In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
train_df = pd.read_csv('../../input/data/train/train_ratings.csv') 
train_df.head(2)

,user,item,time
0,11,4643,1230782529
1,11,170,1230782534


In [3]:
train_df['rating']=1
train_df.head(2)

,user,item,time,rating
0,11,4643,1230782529,1
1,11,170,1230782534,1


# User-based Collaborative Filtering

In [4]:
user_item_matrix = train_df.pivot_table('rating', 'user', 'item').fillna(0)
user_item_matrix=user_item_matrix.astype(int)
user_item_matrix.head(5)

item,1,2,3,4,5,6,7,8,9,10,...,116823,117176,117533,117881,118696,118700,118900,118997,119141,119145
user,,,,,,,,,,,,,,,,,,,,,
11,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
14,1,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
18,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
25,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
31,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [5]:
user_similarity=cosine_similarity(user_item_matrix,user_item_matrix) # 1m
user_similarity_df=pd.DataFrame(user_similarity,index=user_item_matrix.index, columns=user_item_matrix.index)
user_similarity_df.head()

user,11,14,18,25,31,35,43,50,58,60,...,138459,138461,138470,138471,138472,138473,138475,138486,138492,138493
user,,,,,,,,,,,,,,,,,,,,,
11,1.000000,0.134536,0.082279,0.205432,0.203630,0.232003,0.181277,0.215631,0.173518,0.164487,...,0.139679,0.215066,0.124047,0.243275,0.236780,0.168931,0.009262,0.286391,0.081301,0.296853
14,0.134536,1.000000,0.008494,0.125015,0.144150,0.090933,0.106741,0.095893,0.125392,0.103362,...,0.032561,0.139876,0.029881,0.203150,0.081051,0.028172,0.046854,0.140096,0.081349,0.164045
18,0.082279,0.008494,1.000000,0.083624,0.000000,0.147720,0.089018,0.146614,0.070297,0.126428,...,0.159309,0.059406,0.091372,0.011946,0.130808,0.071788,0.051170,0.048682,0.124378,0.102899
25,0.205432,0.125015,0.083624,1.000000,0.042237,0.215814,0.177418,0.292209,0.188112,0.174445,...,0.128225,0.207653,0.112066,0.153846,0.240651,0.145279,0.009414,0.241815,0.177972,0.230717
31,0.203630,0.144150,0.000000,0.042237,1.000000,0.153609,0.104909,0.060475,0.049707,0.156447,...,0.028162,0.117618,0.053841,0.304104,0.175254,0.060914,0.000000,0.082615,0.019544,0.163711


In [8]:
u_sim_top=pd.DataFrame()
TOP_N=50
for user in user_similarity_df.index:  # 3m 38s
    temp=pd.DataFrame(user_similarity_df[user].sort_values(ascending=False)[1:TOP_N+1].index,index=None).T # 0번째는 자기 자신인 1.0이라 뺌
    u_sim_top=pd.concat([u_sim_top, temp],axis=0)
u_sim_top.index=user_similarity_df.index
u_sim_top.head() # user별로 행 # 첫 번째 행에 있는 11번 유저와 가장 비슷한 순서대로 열 0,1,2...

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
user,,,,,,,,,,,,,,,,,,,,,
11,57830,127303,8979,80864,17357,70143,26007,57131,129025,120918,...,31760,5995,59761,46844,102895,49662,74201,85942,82251,133285
14,47208,94619,77003,72851,65537,10620,54351,128298,43426,11424,...,135786,12696,131762,87816,136995,23178,74028,25614,31113,39760
18,33906,112729,67318,76241,113430,127343,56149,116597,11849,99772,...,48633,66002,24604,17782,75223,54825,65830,88484,87196,86752
25,121062,61851,103786,78560,95440,75244,107485,112936,41235,71824,...,49558,50500,45124,92674,70106,7061,17394,36707,136160,110545
31,25444,18389,59090,14989,70046,71961,5622,115434,46283,79812,...,112375,95971,134135,16130,102954,73099,13487,97750,72381,26001


### 비슷한 유저들이 본 아이템 voting

In [ ]:
user_item_count=user_item_matrix.copy()

def to_minus(x) : # 이미 본 아이템은 추천 안 하게 하기 위해서 음수로 설정 
    if x != 0 :
        return -TOP_N
    else : return 0

user_item_count = user_item_count.applymap(to_minus)
user_item_count_cp=user_item_count.copy()
user_item_count.head()

In [ ]:
u_sim_top_np=u_sim_top.to_numpy()
user_item_matrix_np=user_item_matrix.to_numpy()
user_item_count_np=user_item_count_cp.to_numpy()
users = u_sim_top.index
items=user_item_matrix.columns

for i,user in enumerate(users): 
    top_per_user=u_sim_top_np[i,:]
    for top in top_per_user:
        for j,item in enumerate(user_item_matrix_np[np.array(np.where(users == top))[0][0],:]) :
            if item==1:
                user_item_count_np[i,j]+=1
    if i%100==0:
        print(i,"번째 유저까지 count 완료")

In [ ]:
result=[]
for user in user_item_count.index:
    item=user_item_count.loc[user].idxmax()
    result.append([user,item])
    user_item_count.loc[user,item]=0 # 추천했으니까 빼줌
print(result[:5])

In [ ]:
pd.DataFrame(result, columns=["user", "item"]).to_csv("submission.csv", index=False)

# Item-based Collaborative Filtering

In [7]:
item_user_matrix=train_df.pivot_table('rating', 'item', 'user').fillna(0)
item_user_matrix=item_user_matrix.astype(int)
item_user_matrix.head(5)

user,11,14,18,25,31,35,43,50,58,60,...,138459,138461,138470,138471,138472,138473,138475,138486,138492,138493
item,,,,,,,,,,,,,,,,,,,,,
1,1,1,0,0,0,0,0,0,1,0,...,0,0,0,1,0,1,0,1,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
item_similarity=cosine_similarity(item_user_matrix,item_user_matrix) #numpy array
np.fill_diagonal(item_similarity,0) # 대각 행렬 0 으로 채움
item_similarity_df=pd.DataFrame(item_similarity,index=item_user_matrix.index, columns=item_user_matrix.index)
item_similarity_df.head()

item,1,2,3,4,5,6,7,8,9,10,...,116823,117176,117533,117881,118696,118700,118900,118997,119141,119145
item,,,,,,,,,,,,,,,,,,,,,
1,0.000000,0.355963,0.156952,0.030353,0.142656,0.324064,0.151393,0.041788,0.042318,0.355299,...,0.063716,0.053447,0.027279,0.020547,0.067714,0.034473,0.031536,0.036384,0.047508,0.038927
2,0.355963,0.000000,0.150188,0.026293,0.161128,0.179201,0.138460,0.052260,0.046690,0.270732,...,0.045664,0.032694,0.007798,0.011188,0.059993,0.030501,0.022259,0.035864,0.042146,0.025379
3,0.156952,0.150188,0.000000,0.039402,0.174753,0.124785,0.139721,0.042621,0.040891,0.143769,...,0.013331,0.002692,0.005564,0.000000,0.016963,0.005023,0.000000,0.000000,0.003342,0.000000
4,0.030353,0.026293,0.039402,0.000000,0.056504,0.027695,0.037765,0.000000,0.000000,0.018635,...,0.027538,0.011122,0.000000,0.000000,0.030036,0.020752,0.019687,0.021148,0.000000,0.000000
5,0.142656,0.161128,0.174753,0.056504,0.000000,0.072467,0.163124,0.047538,0.025338,0.108791,...,0.014869,0.009008,0.006207,0.013357,0.021623,0.000000,0.000000,0.028546,0.011182,0.009323


In [ ]:
user_item_matrix_np=user_item_matrix.to_numpy()

users = item_user_matrix.columns
items=item_user_matrix.index
result=[]

for i,user in enumerate(users):  #2h이상
    per_user=user_item_matrix_np[i] #i번째 유저가 본 item들에 대한 vector를 per_user에 저장
    watched_items=[]
    for j,item in enumerate(per_user): # 해당 사용자가 본 아이템들 id값 watched_items에 저장 #per_user=[1,0,1]이라면 watched_items=[0,2]기 저장되게
        if item==1:
            watched_items.append(items[j])

    tmp_similarity=item_similarity_df.loc[watched_items,items]
    tmp_similarity= tmp_similarity.to_numpy()
    for _ in range(10): #top 10
        indices=np.where(tmp_similarity==np.max(tmp_similarity))
        rec_item=items[indices[1][0]]
        result.append([user,rec_item])
        tmp_similarity[:,indices[1][0]]=0 # 추천한 아이템에 대한 similarity는 0으로 만들어서 추천대상에서 제외
        

    if i%100==0:
        print(i,"번째 유저까지 count 완료")

In [11]:
pd.DataFrame(result, columns=["user", "item"]).to_csv("submission_item_based.csv", index=False)